## Initial Setting
* Importing packages and dataset

In [14]:
import os
import pandas as pd
import numpy as np

# Load classified dataset
path_raw = os.path.join('..', 'data', 'class_analysis_raw_data.csv')
df_raw = pd.read_csv(path_raw)

---

## Basic Info

 > To protect privacy, all personally identifiable information related to classes and teachers has been removed and replaced with sequential anonymous IDs, in accordance with data privacy guidelines.
 
 * 401 rows x 14 columns
 * **Class-level** activity records from a Junyi's Cognitive Diagnosis system
 * A single teacher may manage multiple classes, resulting in repeated teacher IDs across rows


In [15]:
df_raw.head()

,class_id,class_school,class_city,class_grade_inferred,class_total_missions,class_total_tasks,class_total_students,class_first_activity_date,class_last_activity_date,class_avg_quiz_seconds_taken,class_has_multi_week_activity,class_avg_completion_rate,class_distinct_task_names_by_id,class_peak_activity_slot
0,class_1,NaN,臺北市,NaN,1,1,147,2025-03-26 06:57:00.000000 UTC,2025-03-26 07:01:00.000000 UTC,19.236600,False,0.836735,四上［同分母分數的比較］,上課時間
1,class_2,NaN,宜蘭縣,NaN,1,1,131,2025-03-12 00:37:00.000000 UTC,2025-03-14 03:59:00.000000 UTC,24.635638,False,0.190840,三上［認識分數],上課時間
2,class_3,NaN,宜蘭縣,NaN,1,1,105,2025-03-12 00:37:00.000000 UTC,2025-03-13 03:48:00.000000 UTC,27.179420,False,0.361905,四上［同分母分數的比較］,上課時間
3,class_4,市立銘傳國中,基隆市,NaN,1,1,100,2025-02-04 01:57:00.000000 UTC,2025-02-04 01:57:00.000000 UTC,NaN,False,0.000000,三上［認識分數],無資料
4,class_5,Future實驗教育機構,新北市,NaN,1,1,58,2025-02-06 06:50:00.000000 UTC,2025-02-06 06:52:00.000000 UTC,9.218950,False,0.913793,三上［認識分數],上課時間


In [16]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   class_id                         401 non-null    object 
 1   class_school                     330 non-null    object 
 2   class_city                       401 non-null    object 
 3   class_grade_inferred             257 non-null    float64
 4   class_total_missions             401 non-null    int64  
 5   class_total_tasks                401 non-null    int64  
 6   class_total_students             401 non-null    int64  
 7   class_first_activity_date        401 non-null    object 
 8   class_last_activity_date         401 non-null    object 
 9   class_avg_quiz_seconds_taken     334 non-null    float64
 10  class_has_multi_week_activity    401 non-null    bool   
 11  class_avg_completion_rate        401 non-null    float64
 12  class_distinct_task_na

In [18]:
df_raw.describe()

,class_grade_inferred,class_total_missions,class_total_tasks,class_total_students,class_avg_quiz_seconds_taken,class_avg_completion_rate
count,257.000000,401.000000,401.000000,401.000000,334.000000,401.000000
mean,4.066148,1.443890,1.294264,11.304239,36.033057,0.582737
std,1.494620,0.978503,0.623051,14.983064,23.994057,0.390943
min,0.000000,1.000000,1.000000,1.000000,2.666667,0.000000
25%,3.000000,1.000000,1.000000,3.000000,20.552500,0.200000
50%,4.000000,1.000000,1.000000,7.000000,30.766667,0.700000
75%,5.000000,2.000000,1.000000,15.000000,45.655000,1.000000
max,12.000000,11.000000,4.000000,147.000000,197.404000,1.000000


---

## Data Cleaning

### Columns
* Selected features and binning strategies were collaboratively defined with the product and frontline educators, ensuring alignment with real-world classroom scenarios
    * Mapping: `class_peak_activity_slot`, `class_grade_inferred`, `class_distinct_task_names_by_id`
    * Custom Binning:  `class_total_students`, `class_total_missions`

In [19]:
# Feature Mapping
# Mapping: Class peak activity slot
slot_map_custom = {
    '上課時間': 'During Class',
    '課後輔導': 'After-school Tutoring',
    '無資料': 'Others',
    '放學後': 'Others',
    '早自習': 'Morning Self-study',
    '學前': 'Others'
}
df_raw['class_peak_activity_slot_customized'] = df_raw['class_peak_activity_slot'].map(slot_map_custom)

# Mapping: Grade inference
grade_mapping = {
    '0': 'No Data',
    '1': 'Grade 1',
    '2': 'Grade 2',
    '3': 'Grade 3',
    '4': 'Grade 4',
    '5': 'Grade 5',
    '6': 'Grade 6',
    '7': 'Grade 7',
    '8': 'Grade 8',
    '9': 'Grade 9',
    '10':'Grade 10',
    '11':'Grade 11',
    '12':'Grade 12'
}
df_raw['group_grade_inferred'] = (
    df_raw['class_grade_inferred']
    .fillna(0).astype(int).astype(str)
    .map(grade_mapping)
)

# Mapping: Task name groups (根據課綱或教材邏輯進行群組)
task_mapping = {
    "三上［認識分數]&四上［同分母分數的比較］": "Grade 3 + Grade 4 (Comparison)",
    "三上［認識分數]": "Grade 3",
    "四上［同分母分數的比較］": "Grade 4 (Comparison)",
    "四上［同分母分數的比較］&四上［同分母分數加減］": "Grade 4 Combo (Comparison + Add/Sub)",
    "五上［異分母分數的比較與加減］": "Grade 5",
    "三上［認識分數]&四上［同分母分數的比較］&四上［同分母分數加減］": "Grade 3 + Grade 4 Combo (Comparison + Add/Sub)",
    "四上［同分母分數加減］&五上［異分母分數的比較與加減］": "Grade 4 (Add/Sub) + Grade 5",
    "三上［認識分數]&四上［同分母分數的比較］&五上［異分母分數的比較與加減］": "Grade 3 + Grade 4 (Comparison) + Grade 5",
    "四上［同分母分數加減］": "Grade 4 (Add/Sub)",
    "四上［同分母分數的比較］&四上［同分母分數加減］&五上［異分母分數的比較與加減］": "Grade 4 Combo + Grade 5",
    "三上［認識分數]&四上［同分母分數加減］": "Grade 3 + Grade 4 (Add/Sub)",
    "四上［同分母分數的比較］&五上［異分母分數的比較與加減］": "Grade 4 (Comparison) + Grade 5",
    "三上［認識分數]&四上［同分母分數的比較］&四上［同分母分數加減］&五上［異分母分數的比較與加減］": "Grade 3 + Grade 4 Combo + Grade 5",
    "三上［認識分數]&五上［異分母分數的比較與加減］": "Grade 3 + Grade 5"
}
df_raw['group_task_name'] = df_raw['class_distinct_task_names_by_id'].map(task_mapping)

# Custom Binning
# Binning: Total students per class
bins_students = [0, 1, 6, 11, 21, np.inf]
labels_students = ['0-1', '1-5', '6-10', '11-20', '21+']
df_raw['group_total_students'] = pd.cut(
    df_raw['class_total_students'], 
    bins=bins_students, 
    labels=labels_students, 
    right=False
)

# Binning: Total missions
bins_missions = [1, 2, 3, 4, np.inf]
labels_missions = ['1-1.99', '2-2.99', '3-3.99', '4+']
df_raw['group_total_missions'] = pd.cut(
    df_raw['class_total_missions'], 
    bins=bins_missions, 
    labels=labels_missions, 
    right=False
)

df_raw.head()

,class_id,class_school,class_city,class_grade_inferred,class_total_missions,class_total_tasks,class_total_students,class_first_activity_date,class_last_activity_date,class_avg_quiz_seconds_taken,class_has_multi_week_activity,class_avg_completion_rate,class_distinct_task_names_by_id,class_peak_activity_slot,class_peak_activity_slot_customized,group_grade_inferred,group_task_name,group_total_students,group_total_missions
0,class_1,NaN,臺北市,NaN,1,1,147,2025-03-26 06:57:00.000000 UTC,2025-03-26 07:01:00.000000 UTC,19.236600,False,0.836735,四上［同分母分數的比較］,上課時間,During Class,No Data,Grade 4 (Comparison),21+,1-1.99
1,class_2,NaN,宜蘭縣,NaN,1,1,131,2025-03-12 00:37:00.000000 UTC,2025-03-14 03:59:00.000000 UTC,24.635638,False,0.190840,三上［認識分數],上課時間,During Class,No Data,Grade 3,21+,1-1.99
2,class_3,NaN,宜蘭縣,NaN,1,1,105,2025-03-12 00:37:00.000000 UTC,2025-03-13 03:48:00.000000 UTC,27.179420,False,0.361905,四上［同分母分數的比較］,上課時間,During Class,No Data,Grade 4 (Comparison),21+,1-1.99
3,class_4,市立銘傳國中,基隆市,NaN,1,1,100,2025-02-04 01:57:00.000000 UTC,2025-02-04 01:57:00.000000 UTC,NaN,False,0.000000,三上［認識分數],無資料,Others,No Data,Grade 3,21+,1-1.99
4,class_5,Future實驗教育機構,新北市,NaN,1,1,58,2025-02-06 06:50:00.000000 UTC,2025-02-06 06:52:00.000000 UTC,9.218950,False,0.913793,三上［認識分數],上課時間,During Class,No Data,Grade 3,21+,1-1.99


* Based on insights from educational professionals, classroom activity patterns are best interpreted by first considering **the time slot of use**, followed by **the class size**. Therefore, the combined feature `customized_slot_and_student` is constructed in that specific order.

In [20]:
# Combined Columns
df_raw['cutomized_slot_and_student'] = (
    df_raw['class_peak_activity_slot_customized'].astype(str) + '_' +
    df_raw['group_total_students'].astype(str)
)

df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   class_id                             401 non-null    object  
 1   class_school                         330 non-null    object  
 2   class_city                           401 non-null    object  
 3   class_grade_inferred                 257 non-null    float64 
 4   class_total_missions                 401 non-null    int64   
 5   class_total_tasks                    401 non-null    int64   
 6   class_total_students                 401 non-null    int64   
 7   class_first_activity_date            401 non-null    object  
 8   class_last_activity_date             401 non-null    object  
 9   class_avg_quiz_seconds_taken         334 non-null    float64 
 10  class_has_multi_week_activity        401 non-null    bool    
 11  class_avg_completio

### Missing Value
* Since **Grade 0** does not exist in the real-world education system, we treat it as a placeholder for missing or unidentifiable values.
* To ensure more accurate analysis, we exclude classes where the    **inferred grade** equals 0 .
* 255 rows remained after dropping missing values.

In [21]:
df_cleaned = df_raw[df_raw['group_grade_inferred'] != 'No Data']
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255 entries, 8 to 399
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   class_id                             255 non-null    object  
 1   class_school                         223 non-null    object  
 2   class_city                           255 non-null    object  
 3   class_grade_inferred                 255 non-null    float64 
 4   class_total_missions                 255 non-null    int64   
 5   class_total_tasks                    255 non-null    int64   
 6   class_total_students                 255 non-null    int64   
 7   class_first_activity_date            255 non-null    object  
 8   class_last_activity_date             255 non-null    object  
 9   class_avg_quiz_seconds_taken         228 non-null    float64 
 10  class_has_multi_week_activity        255 non-null    bool    
 11  class_avg_completion_rat

---

## Feature Engineering


### One-Hot Encoding
* Categorical variables are transformed using **one-hot encoding** to ensure compatibility with machine learning models that require numerical inputs.
* 43 columns after one-hot encoding

In [22]:
# Select features based on product team
feature_cols = [
      'cutomized_slot_and_student',
      'group_grade_inferred',
      'group_task_name',
      'group_total_missions'
]

df_clustering = df_cleaned[feature_cols]
df_encoded = pd.get_dummies(df_clustering)

df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255 entries, 8 to 399
Data columns (total 43 columns):
 #   Column                                                          Non-Null Count  Dtype
---  ------                                                          --------------  -----
 0   cutomized_slot_and_student_After-school Tutoring_1-5            255 non-null    bool 
 1   cutomized_slot_and_student_After-school Tutoring_11-20          255 non-null    bool 
 2   cutomized_slot_and_student_After-school Tutoring_21+            255 non-null    bool 
 3   cutomized_slot_and_student_After-school Tutoring_6-10           255 non-null    bool 
 4   cutomized_slot_and_student_During Class_1-5                     255 non-null    bool 
 5   cutomized_slot_and_student_During Class_11-20                   255 non-null    bool 
 6   cutomized_slot_and_student_During Class_21+                     255 non-null    bool 
 7   cutomized_slot_and_student_During Class_6-10                    255 non-null

---

## Exported Processed Dataset

In [23]:
# Exported Cleaned & Encoded Dataset
path_cleand_class = os.path.join('..', 'data', 'cleaned_class.csv')
df_cleaned.to_csv(path_cleand_class, index=False)

path_encoded_class = os.path.join('..', 'data', 'encoded_class.csv')
df_encoded.to_csv(path_encoded_class, index=False)